In [110]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

ohe = OneHotEncoder()
sc = StandardScaler()

df_train = pd.read_csv('./train-nn.csv').dropna()

C:\Users\nikol\AppData\Local\Temp\ipykernel_14816\1649306298.py:11: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('./train.csv').dropna()


In [47]:
# Day of week one-hot encoding

def get_weekday_one_hot(week_day):
    week_days = range(1, 8)
    week_days_one_hot = []

    for value in week_days:
        day = [0 for _ in range(len(week_days))]
        day[value - 1] = 1
        week_days_one_hot.append(day)

    return week_days_one_hot[week_day - 1]

In [33]:
# Time difference between current and previous event

# Time difference between previous and second previous event

# Is in working time

event_types = df_train['event'].unique()
event_types

array(['A_PREACCEPTED', 'W_Completeren aanvraag', 'A_ACCEPTED',
       'O_SELECTED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes',
       'O_SENT_BACK', 'W_Valideren aanvraag', 'A_SUBMITTED',
       'A_PARTLYSUBMITTED', 'A_FINALIZED', 'O_CANCELLED', 'O_ACCEPTED',
       'A_REGISTERED', 'A_APPROVED', 'W_Afhandelen leads', 'A_DECLINED',
       'W_Nabellen incomplete dossiers', 'A_ACTIVATED', 'O_DECLINED',
       'W_Beoordelen fraude', 'A_CANCELLED'], dtype=object)

In [48]:
df_train['weekday'] = df_train['weekday'].copy().transform(lambda x: get_weekday_one_hot(int(x)))
df_train['weekday_previous'] = df_train['weekday_previous'].copy().transform(lambda x: get_weekday_one_hot(int(x)))
df_train['weekday_2previous'] = df_train['weekday_2previous'].copy().transform(lambda x: get_weekday_one_hot(int(x)))

In [111]:
df_train['total_milliseconds_td1'] = df_train['time_diff1'] / pd.to_timedelta(1, unit='ms')
df_train['total_milliseconds_td2'] = df_train['time_diff2'] / pd.to_timedelta(1, unit='ms')
df_train['is_work_day'] = df_train['is_work_day'].apply(lambda x: float(x))

C:\Users\nikol\AppData\Local\Temp\ipykernel_14816\762805595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['total_milliseconds_td1'] = df_train['time_diff1'] / pd.to_timedelta(1, unit='ms')
C:\Users\nikol\AppData\Local\Temp\ipykernel_14816\762805595.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['total_milliseconds_td2'] = df_train['time_diff2'] / pd.to_timedelta(1, unit='ms')
C:\Users\nikol\AppData\Local\Temp\ipykernel_14816\762805595.py:3: SettingWithCopyWarning: 
A value is tr

In [112]:
df_train['event'] = df_train['event'].apply(lambda x: list(event_types).index(x))
df_train['second_previous'] = df_train['second_previous'].apply(lambda x: list(event_types).index(x))
df_train['previous'] = df_train['previous'].apply(lambda x: list(event_types).index(x))

In [113]:
df_train['weekday']

2        5.0
3        5.0
4        5.0
5        5.0
6        5.0
        ... 
34772    0.0
34773    0.0
34774    0.0
34775    0.0
34776    0.0
Name: weekday, Length: 34775, dtype: float64

In [129]:
# features = ['weekday', 'weekday_previous', 'weekday_2previous', 'time_diff1', 'time_diff2', 'is_work_day', 'previous', 'second_previous', 'event']
features = ['total_milliseconds_td1', 'total_milliseconds_td2', 'is_work_day']
features1 = ['previous', 'second_previous', 'event']
features2 = ['weekday', 'weekday_previous', 'weekday_2previous']

x = df_train[features].values
x1 = df_train[features1].values
x2 = df_train[features2].values

df_train_y = pd.to_datetime(df_train['time_next_event'])

y = df_train_y.transform(lambda x: x.timestamp()).values
# y = y.reshape(-1,1)
x2.shape

(34775, 3)

In [115]:
inputs = keras.Input(shape=(3, ), name='input')
inputs1 = keras.Input(shape=(3, ), name='input1')
inputs2 = keras.Input(shape=(3, ), name='input2')

merged = keras.layers.Concatenate(axis=1)([inputs, inputs1, inputs2])

hidden = layers.Dense(4, input_dim=3, activation='sigmoid', name='first')(merged)
hidden1 = layers.Dense(6, activation='sigmoid', name='second')(hidden)
hidden2 = layers.Dense(5, activation='sigmoid', name='third')(hidden1)
output = layers.Dense(1, activation='linear', name='output')(hidden2)

model = keras.Model(
    inputs = [inputs, inputs1, inputs2],
    outputs = output,
)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.summary()

model.fit([x, x1, x2], epochs=500, batch_size=128, y=y)

Model: "model_34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 3)]          0           []                               
                                                                                                  
 input1 (InputLayer)            [(None, 3)]          0           []                               
                                                                                                  
 input2 (InputLayer)            [(None, 3)]          0           []                               
                                                                                                  
 concatenate_31 (Concatenate)   (None, 9)            0           ['input[0][0]',                  
                                                                  'input1[0][0]',          

In [124]:
x.tolist()

[[53026.0, 334.0, 0.0],
 [0.0, 53026.0, 0.0],
 [39428531.0, 0.0, 1.0],
 [-150609.0, 39428531.0, 1.0],
 [145935.0, -150609.0, 1.0],
 [1954.0, 145935.0, 1.0],
 [183.0, 1954.0, 1.0],
 [1829910.0, 183.0, 1.0],
 [619788796.0, 1829910.0, 1.0],
 [154821609.0, 619788796.0, 1.0],
 [-2123.0, 154821609.0, 1.0],
 [253943257.0, -2123.0, 1.0],
 [-1045718770.0, 253943257.0, 0.0],
 [3939.0, -1045718770.0, 0.0],
 [54453.0, 3939.0, 0.0],
 [0.0, 54453.0, 0.0],
 [12455745.0, 0.0, 1.0],
 [9884597.0, 12455745.0, 1.0],
 [-2775.0, 9884597.0, 1.0],
 [1567.0, -2775.0, 1.0],
 [181135674.0, 1567.0, 1.0],
 [-3464.0, 181135674.0, 1.0],
 [0.0, -3464.0, 1.0],
 [1882.0, 0.0, 1.0],
 [32.0, 1882.0, 1.0],
 [0.0, 32.0, 1.0],
 [580645182.0, 0.0, 1.0],
 [-1739.0, 580645182.0, 1.0],
 [0.0, -1739.0, 1.0],
 [3472837.0, 0.0, 1.0],
 [9647231.0, 3472837.0, 1.0],
 [192912.0, 9647231.0, 1.0],
 [1.0, 192912.0, 1.0],
 [-799616020.0, 1.0, 0.0],
 [304.0, -799616020.0, 0.0],
 [42435.0, 304.0, 0.0],
 [191393600.0, 42435.0, 1.0],
 [9505.0

In [134]:
inputs = keras.Input(shape=(3, 1), name='input')
inputs1 = keras.Input(shape=(3, 1), name='input1')

merged = keras.layers.Concatenate(axis=1)([inputs, inputs1])

hidden = layers.Dense(4, activation='sigmoid', name='first')(merged)
hidden1 = layers.Dense(6, activation='sigmoid', name='second')(hidden)
hidden2 = layers.Dense(5, activation='sigmoid', name='third')(hidden1)
output = layers.Dense(1, activation='sigmoid', name='output')(hidden2)

model = keras.Model(
    inputs = [inputs, inputs1],
    outputs = output,
)

inputs2 = keras.Input(shape=(3, 7), name='input2')

hidden_second = layers.Dense(4, activation='sigmoid', name='first1')(inputs2)
output_second = layers.Dense(1, activation='sigmoid', name='output1')(hidden_second)

model_second = keras.Model(inputs=inputs2, outputs=output_second)

combined = keras.layers.Concatenate(axis=1)([model.output, model_second.output])

hidden_third = layers.Dense(2, activation="sigmoid", name='first2')(combined)
output_third = layers.Dense(1, activation='linear', name='output_final')(hidden_third)

model_final = keras.Model(inputs=[model.input, model_second.input], outputs=output_third)

model_final.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model_final.summary()

model_final.fit([x, x1, x2.tolist()], epochs=10, batch_size=128, y=y.tolist())

Model: "model_63"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 3, 1)]       0           []                               
                                                                                                  
 input1 (InputLayer)            [(None, 3, 1)]       0           []                               
                                                                                                  
 concatenate_52 (Concatenate)   (None, 6, 1)         0           ['input[0][0]',                  
                                                                  'input1[0][0]']                 
                                                                                                  
 first (Dense)                  (None, 6, 4)         8           ['concatenate_52[0][0]']  

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>", '(<class \'list\'> containing values of types {\'(<class \\\'list\\\'> containing values of types {"<class \\\'float\\\'>"})\'})'}), (<class 'list'> containing values of types {"<class 'float'>"})